In [90]:
%%time
import pal
import json
import random
import io
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 165 μs, sys: 4 μs, total: 169 μs
Wall time: 200 μs


In [98]:
#(4) REMOVE_START[position]REMOVE_END: REMOVE takes a position and removes the word at that position. NOTE: This list is zero indexed, so REMOVE_START[0]REMOVE_END would remove the first element of the list.

prompt = """Whenever you synthesize information and perform reasoning, it is a Thought. When you need to perform a function on the list, it is an Action. There are 4 types of actions: 
(1) ADD_START[word;number]ADD_END: ADD takes in a word and a number, and appends the word the number of times that it's specified. For example, ADD[hello;3] will append "hello" 3 times to the list.
(2) INSERT_START[word;position]INSERT_END: INSERT takes in a word and a position, and inserts that word at the position. For example, if our list is ["hello", "hi", "three", "four", "five"], then INSERT[bye;1 would turn the list into ["hello", "bye", "hi", "three", "four", "five"].
(3) PRINT_START[]PRINT_END: PRINT will print all the words in the list with spaces in between. For example, if the current list is ["hello", "world", "hello"], then PRINT_START[]PRINT_END will return "hello world hello". 
(4) FINISH_START[answer]FINISH_END: When you finish a question, write your answer as FINISH_START[answer]FINISH_END and terminate your response.
Terminate your response after an action. An Observation will be returned by the user. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called FINISH.  Let's think step by step. Think carefully about the question to accurately perform actions. The questions may be vaguely phrased, some advice is to take it as literally as possible. REMEMBER: The list (and therefore the positions of the words) change after insertion. Take this into account for your inputs. For example, if we inserted at position 2, what was previously position 4 is now position 5."""
repeat_messages = [
    {"role": "system", "content": "You are a helpful assistant that accurately solves reasoning questions by acting as an agent who will perform Thoughts and Actions."},
    {"role": "user", "content": prompt + '\n' + "Question: Output hello the world the eat 3 times, but after the 2nd time, say meow"},
    {"role": "assistant", "content": """Thought: First, I will add "hello the world the eat" 3 times to the list with the ADD action.
Action: ADD_START[hello the world the eat;3]"""},
    {"role": "user", "content": """Observation: Added "hello the world the eat" 3 times."""},
    {"role": "assistant", "content": """Thought: Now I need to insert "meow" into the position 2 to be after the 2nd "hello world".
Action: INSERT[meow;2]"""},
    {"role": "user", "content": """Observation: Inserted "meow" at position 2."""},
    {"role": "assistant", "content": """Thought: Now to see the final list, I will perform the action PRINT.
Action: PRINT_START[]PRINT_END"""},
    {"role": "user", "content": "Observation: hello the world the eat hello the world the eat meow hello the world the eat"},
    {"role": "assistant", "content": """Thought: Now I can finish with this list.
Action: FINISH_START[hello the world the eat hello the world the eat meow hello the world the eat]"""}
]

In [100]:
interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)
json_list = None
max_count = 3
counter = 0
responses = []
with open('../prompt-lib/data/tasks/repeatcopy.jsonl', 'r') as json_file:
    json_list = list(json_file)
jsonSamples = random.sample(range(0, len(json_list)), max_count)
maxIters = 20
print(jsonSamples)

for json_pos in tqdm(range(len(json_list)), desc="Going through math questions"):
    result = json.loads(json_list[json_pos])
    question = result["input"]
    target = result["target"]
    curResponse = {"question": question, "target": target}
    messages = []
    for message in repeat_messages:
        messages.append(message)
    messages.append({"role": "user", "content": prompt + '\n' + "Question: " + question})
    # interface.verbose = True
    finished = False
    curList = []
    #print("Question: ")
    #print(question)
    #print("")
    numIters = 0
    while not finished:
        if numIters >= maxIters:
            print("##############################################")
            print("Maximum Number of Iterations Reached")
            print("Question " + str(json_pos))
            print("")
            curResponse["received"] = -987654321
            break
        numIters += 1
        gens = interface.generate_history(messages=messages, max_tokens=1024)
        output = gens[0]
        #print("Output:")
        #print(output)
        #print("")
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        if "FINISH" in curAction:
            finished = True
            expression = curAction[curAction.find("FINISH_START[") + len("FINISH_START["):curAction.find("]FINISH_END")]
            try:
                curResponse["received"] = expression
            except:
                print("#################################################")
                print("Finish Error")
                print("Question " + str(counter))
                print(output)
                curResponse["received"] = -987654321
        else:
            evaled = ""
            if "ADD" in curAction:
                expression = curAction[curAction.find("ADD_START[") + len("ADD_START["):curAction.find("]ADD_END")]
                expression = expression.split(';')
                try:
                    for i in range(int(expression[1])):
                        curList.append(expression[0])
                    evaled = "Added \"" + expression[0] + "\" " + expression[1] + " times.\n"
                    evaled = evaled + "This is the new list: " + str(curList)
                except Exception as e:
                    evaled = f"That was an invalid input. This was the exception: {e}. Try again."
                #print(curTable)
            elif "INSERT" in curAction:
                expression = curAction[curAction.find("INSERT_START[") + len("INSERT_START["):curAction.find("]INSERT_END")]
                expression = expression.split(';')
                try:
                    #posits = json.loads(expression[1])
                    #posits.sort()
                    posits = [int(expression[1])]
                    for i in range(len(posits)):
                        curList.insert(i + posits[i], expression[0])
                    evaled = "Inserted \"" + expression[0] + "\" at position"
                    for posit in posits:
                        evaled = evaled + ' ' + str(posit)
                    evaled = evaled + '\n'
                    evaled = evaled + "This is the new list: " + str(curList)
                except Exception as e:
                    evaled = f"That was an invalid input. This was the exception: {e}. Try again."
            elif "REMOVE" in curAction:
                try:
                    expression = curAction[curAction.find("REMOVE_START[") + len("REMOVE_START["):curAction.find("]REMOVE_END")]
                    curList.pop(int(expression))
                    evaled = "Removed word at position " + expression + '\n'
                    evaled = evaled + "This is the new list: " + str(curList)
                except Exception as e:
                    evaled = f"That was an invalid input. This was the exception: {e}. Try again."
            elif "PRINT" in curAction:
                expression = curAction[curAction.find("PRINT_START[") + len("PRINT_START["):curAction.find("]PRINT_END")]
                evaled = " ".join(curList)
            else:
                evaled = "That was not a valid calculation. Please input a valid one."
        messages.append({"role": "user", "content": "Observation: " + evaled})
    curResponse["messages"] = messages
    responses.append(curResponse)
numCorrect = 0
for i in range(len(responses)):
    response = responses[i]
    if response["target"] == response["received"]:
        numCorrect += 1
    else:
        print(i)
        print("Target: " + str(response["target"]))
        print("Received: " + str(response["received"]))
        print("")
print("Number correct = " + str(numCorrect))

[21, 31, 28]


Going through math questions:   0%|          | 0/32 [00:00<?, ?it/s]

3
Target: all the world all the world is a stage all the world all the world is a stage all the world all the world is a stage all the world
Received: all the world all the world is a stage all the world is a stage all the world is a stage all the world all the world all the world

5
Target: python python data python python data python python data
Received: python python data python python data python python data python python data

6
Target: logic logic logic true logic logic logic logic logic true logic logic logic
Received: logic logic logic logic true logic logic logic logic logic true logic logic

11
Target: carbon pricing is an carbon pricing is an interesting concept carbon pricing is an carbon pricing is an
Received: carbon pricing is an carbon pricing is an carbon pricing is an interesting concept carbon pricing is an

15
Target: election to cool the council cool election to cool the council cool election to cool the council cool
Received: election to the council election to t

In [88]:
for message in responses[3]["messages"][len(repeat_messages):]:
    print(message["content"])

Whenever you synthesize information and perform reasoning, it is a Thought. When you need to perform a function on the list, it is an Action. There are 4 types of actions: 
(1) ADD_START[word;number]ADD_END: ADD takes in a word and a number, and appends the word the number of times that it's specified. For example, ADD[hello;3] will append "hello" 3 times to the list.
(2) INSERT_START[word;position]INSERT_END: INSERT takes in a word and a position, and inserts that word at the position. For example, if our list is ["hello", "hi", "three", "four", "five"], then INSERT[bye;1 would turn the list into ["hello", "bye", "hi", "three", "four", "five"].
(3) PRINT_START[]PRINT_END: PRINT will print all the words in the list with spaces in between. For example, if the current list is ["hello", "world", "hello"], then PRINT_START[]PRINT_END will return "hello world hello". 
(4) FINISH_START[answer]FINISH_END: When you finish a question, write your answer as FINISH_START[answer]FINISH_END and term

In [61]:
print(len(json_list))

32


In [14]:
tempList = []
for i in range(int("5")):
    tempList.append("cat")
posits = json.loads("[1, 3, 5, 7, 9]")
posits.sort()
print(tempList)
for i in range(len(posits)):
    tempList.insert(i + posits[i], "mouse")
    print(tempList)

['cat', 'cat', 'cat', 'cat', 'cat']
['cat', 'mouse', 'cat', 'cat', 'cat', 'cat']
['cat', 'mouse', 'cat', 'cat', 'mouse', 'cat', 'cat']
['cat', 'mouse', 'cat', 'cat', 'mouse', 'cat', 'cat', 'mouse']
['cat', 'mouse', 'cat', 'cat', 'mouse', 'cat', 'cat', 'mouse', 'mouse']
['cat', 'mouse', 'cat', 'cat', 'mouse', 'cat', 'cat', 'mouse', 'mouse', 'mouse']
